In [9]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.constraints import maxnorm
from keras.optimizers import SGD

from IPython.display import SVG,display
from keras.utils.vis_utils import model_to_dot

batch_size = 32
num_classes = 11
epochs = 15

In [2]:
def hot(y):
    label_binarizer = LabelBinarizer()
    label_binarizer.fit(range(11))
    new_y = np.zeros((y.shape[0],y.shape[1]*num_classes))
    for i in range(len(y)):
        new_y[i,:] = label_binarizer.transform(y[i]).flatten()
    return new_y

In [3]:
X = np.load("resizecrop.npy")
y = np.load("nummertrain.npy").astype(int)

In [4]:
y = hot(y)
x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

In [5]:
ynum = y_train[:,55:66]
ynum_t = y_test[:,55:66]
print('x_train shape:', x_train.shape)
print('y_train shape:', ynum.shape)
print(x_train.shape[0], 'train samples')

x_train shape: (26721, 25, 23, 3)
y_train shape: (26721, 11)
26721 train samples


In [8]:
a = Input(shape=x_train.shape[1:])

x = Conv2D(32, (2, 2), padding='same')(a)
x = Activation('relu')(x)
x = Conv2D(32, (2, 2))(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(64, (2, 2), padding='same')(x)
x = Activation('relu')(x)
x = Conv2D(64, (2, 2))(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(64, (4, 4), padding='same')(x)
x = Activation('relu')(x)
x = Conv2D(64, (4, 4))(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

conv_out = Flatten()(x)

num = Dense(128, activation='relu')(conv_out)
num = Dense(128, activation='relu')(num)
num = Dropout(0.5)(num)
numout = Dense(num_classes, activation='softmax',name="num")(num)

model = Model(inputs=a, outputs=numout)


# display(SVG(model_to_dot(model).create(prog='dot', format='svg')))
# load weights
# model.load_weights("weights.forksvhmbest.hdf5")

callbacks = [EarlyStopping(monitor='val_loss', patience=2, verbose=0)]

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'],
              callbacks=callbacks)

In [10]:
model.fit(x_train, ynum,
        batch_size=batch_size,
        epochs=epochs,
        validation_split=0.2,
        shuffle=True,
        verbose=2)

/home/a/workspace/venv/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:2094: UserWarning: Expected no kwargs, you passed 1
kwargs passed to function are ignored with Tensorflow backend
  warnings.warn('\n'.join(msg))


Train on 21376 samples, validate on 5345 samples
Epoch 1/15
14s - loss: 0.9261 - acc: 0.6450 - val_loss: 0.4482 - val_acc: 0.8249
Epoch 2/15
13s - loss: 0.3977 - acc: 0.8567 - val_loss: 0.3052 - val_acc: 0.8889
Epoch 3/15
13s - loss: 0.2850 - acc: 0.9005 - val_loss: 0.3009 - val_acc: 0.8941
Epoch 4/15
13s - loss: 0.2216 - acc: 0.9241 - val_loss: 0.2125 - val_acc: 0.9276
Epoch 5/15
13s - loss: 0.1822 - acc: 0.9379 - val_loss: 0.2200 - val_acc: 0.9227
Epoch 6/15
13s - loss: 0.1521 - acc: 0.9495 - val_loss: 0.1974 - val_acc: 0.9328
Epoch 7/15
13s - loss: 0.1241 - acc: 0.9589 - val_loss: 0.2093 - val_acc: 0.9371
Epoch 8/15
13s - loss: 0.1052 - acc: 0.9656 - val_loss: 0.2390 - val_acc: 0.9259
Epoch 9/15
13s - loss: 0.0866 - acc: 0.9729 - val_loss: 0.2075 - val_acc: 0.9392
Epoch 10/15
13s - loss: 0.0722 - acc: 0.9768 - val_loss: 0.2470 - val_acc: 0.9373
Epoch 11/15
13s - loss: 0.0605 - acc: 0.9811 - val_loss: 0.2537 - val_acc: 0.9426
Epoch 12/15
13s - loss: 0.0529 - acc: 0.9831 - val_loss: 0

In [28]:
score = model.evaluate(x_test, ynum_t, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.save('numtower.h5')

Test loss: 0.281458338438
Test accuracy: 0.938033228398
